In [85]:
import cv2 as cv
import numpy as np
import os
import sklearn
from sklearn.svm import SVC
from skimage.feature import graycomatrix, graycoprops
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV ,StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

import pickle

In [86]:
MASK_PATH = './Dataset/mask/'
NO_MASK_PATH = './Dataset/no_mask/'

MASK_PATH_PREPROCESSED = './PreProcessed/mask/'
NO_MASK_PATH_PREPROCESSED = './PreProcessed/no_mask/'



In [87]:
mask_data = []
no_mask_data = []

for i in os.listdir(MASK_PATH_PREPROCESSED):
    path = os.path.join(MASK_PATH_PREPROCESSED,i)
    img = cv.imread(path,cv.IMREAD_GRAYSCALE)
    mask_data.append(img)
   
for i in os.listdir(NO_MASK_PATH_PREPROCESSED):
    path = os.path.join(NO_MASK_PATH_PREPROCESSED,i)
    img = cv.imread(path,cv.IMREAD_GRAYSCALE)
    no_mask_data.append(img)

In [88]:
masked_features = []
no_masked_features = []

level = 8

for i in mask_data:
    gray = np.floor(i / (256 / level)).astype('uint8')
    glcm = graycomatrix(gray, distances=[1], angles=[0, np.pi/4, np.pi/2, np.pi],levels=level, symmetric=True, normed=True)
    # print(glcm.reshape(1,-1))
    masked_features.append([
            graycoprops(glcm, 'contrast').mean(),
            graycoprops(glcm, 'dissimilarity').mean(),
            graycoprops(glcm, 'homogeneity').mean(),
            graycoprops(glcm, 'energy').mean(),
            graycoprops(glcm, 'correlation').mean()
        ])
    
for i in no_mask_data:
    gray = np.floor(i / (256 / level)).astype('uint8')
    glcm = graycomatrix(gray, distances=[1], angles=[0, np.pi/4, np.pi/2, np.pi],levels=level, symmetric=True, normed=True)
    no_masked_features.append([
            graycoprops(glcm, 'contrast').mean(),
            graycoprops(glcm, 'dissimilarity').mean(),
            graycoprops(glcm, 'homogeneity').mean(),
            graycoprops(glcm, 'energy').mean(),
            graycoprops(glcm, 'correlation').mean()
        ])

In [89]:
print(np.array(masked_features).shape)

(649, 5)


In [90]:
df = pd.DataFrame(masked_features)
df['label'] = 1

df1 = pd.DataFrame(no_masked_features)
df1['label'] = 0

merged = pd.concat([df,df1])
merged

,0,1,2,3,4,label
0,0.619351,0.384239,0.830065,0.407817,0.937481,1
1,0.687188,0.345690,0.854534,0.432327,0.931934,1
2,0.711939,0.433662,0.809139,0.362867,0.936787,1
3,0.994645,0.526109,0.780055,0.364014,0.875685,1
4,0.454090,0.281114,0.875594,0.411168,0.971503,1
...,...,...,...,...,...,...
637,0.323869,0.269497,0.870614,0.335879,0.948311,0
638,0.241062,0.236766,0.882046,0.336501,0.953307,0
639,0.435524,0.371419,0.820615,0.257412,0.953135,0
640,0.628917,0.412432,0.814122,0.293125,0.913998,0


In [91]:
y = merged['label']
x = merged.drop(columns=['label'])

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [92]:
# svm = SVC(kernel='rbf',gamma=0.01, C=0.1)
# svm.fit(X_train,y_train)


# cross_validation = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
# param_grid = {'C': [0.1, 1, 10, 100, 1000],  
#               'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
#               'kernel': ['rbf']}  
  
# grid = GridSearchCV(SVC(), param_grid, cv=cross_validation,refit = True, verbose = 3) 
  
# grid.fit(X_train, y_train) 


In [93]:
# print(grid.best_params_)

In [94]:
classifier = SVC(C= 10, gamma= 0.01, kernel= 'rbf')
classifier.fit(X_train, y_train)

SVC(C=10, gamma=0.01)

In [95]:
# best_model = grid.best_estimator_
# y_pred = best_model.predict(X_test)

ytrain_pred = classifier.predict(X_train)
ytest_pred = classifier.predict(X_test)


# print(f"Best parameters found: {grid.best_params_}")
# print(f"Best cross-validation score: {grid.best_score_}")

print(classification_report(y_train, ytrain_pred))
print(classification_report(y_test, ytest_pred))

              precision    recall  f1-score   support

           0       0.65      0.86      0.74       437
           1       0.79      0.52      0.63       427

    accuracy                           0.70       864
   macro avg       0.72      0.69      0.69       864
weighted avg       0.72      0.70      0.69       864

              precision    recall  f1-score   support

           0       0.58      0.87      0.70       205
           1       0.78      0.42      0.54       222

    accuracy                           0.63       427
   macro avg       0.68      0.64      0.62       427
weighted avg       0.68      0.63      0.62       427



In [96]:

matrix_kebingungan = confusion_matrix(ytest_pred, y_test)
print(matrix_kebingungan)

matrix_kebingungan = confusion_matrix(ytrain_pred, y_train)
print(matrix_kebingungan)

[[178 129]
 [ 27  93]]
[[377 203]
 [ 60 224]]


In [97]:




# img = cv.imread('./b.jpg', cv.IMREAD_GRAYSCALE)
# # img = no_mask_data[-1:]
# # print(img)
# faces = haarcasade.detectMultiScale(img, scaleFactor = 1.2, minNeighbors = 5)
# if not len(faces) < 1:
#     for face_rect in faces:
#         x, y, w, h = face_rect
#         face_image = img[y:y+w, x:x+h]
#         resized = cv.resize(face_image,(128,128))

#         hog_feature = hog(resized, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
#         test_data = pd.DataFrame(hog_feature).T

#         result=best_model.predict(test_data)[0]
#         print(result)
#         if result == 0:
#             result = 'No Mask'
#         else:
#             result = 'Mask'


#         cv.putText(img, result, (100,100), cv.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 0), 1)
#         cv.imshow('',img)
#         cv.waitKey(0)
#         cv.destroyAllWindows()

#         cv.imshow('',resized)
#         cv.waitKey(0)
#         cv.destroyAllWindows()

In [98]:
import pickle
model_filename = './model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(best_model, file)

print(f"Model saved to {model_filename}")

NameError: name 'best_model' is not defined